In [1]:
import IntellivueDecoder
import IntellivueDistiller
import RS232
import time
import datetime
import numpy as np
import logging

class SampledDataBuffer(object):
    # This is a fixed-length double queue for time/value pairs s.t. f(t)=y
    # Once initialized, it can be updated with a single time point and a set of values
    # taken at a given frequency.
    # t is re-evaluated relative to the initialization time

    def __init__(self, freq, dur):
        self.freq = freq
        self.dur = dur
        self.y = np.zeros(self.freq*self.dur)
        self.t = np.zeros(self.freq*self.dur)
        self.start_time = datetime.datetime.now()
        self.t1 = None
        self.dropped_packets = 0

        # self.t = np.linspace(now-self.dur, now-1, self.freq*self.dur)

    def rolling_append(self, _t0, values):

        if values is None:
            return

        # Convert t0 to seconds since start_time
        t0 = (_t0 - self.start_time).total_seconds()
        if not self.t1:
            self.t1 = t0
        # logging.debug('t0 {0}'.format(t0))
        # logging.debug('t1 {0}'.format(self.t1))

        t0 -= self.t1

        length = values.size or 1  # For scalar
        # logging.debug('len {0}'.format(length))
        if length == 1:
            times = t0
        else:
            sec_offset = length/self.freq
            times = np.linspace(t0, t0+sec_offset, length)

        # TODO: Fix this if it's important; it needs to fix itself so it does't get into a loop
        # if t0 - self.t[-1] > 0:
        #     self.dropped_packets += 1
        #     logging.warn('>{0} dropped packets (t0={t0} != t[-1]={t1})'.format(self.dropped_packets, t0=t0, t1=self.t[-1]))
        #     # Everytime this happens, it increases the total duration; if it's consistent, it
        #     # will be a multiplier on the duration.

        self.y = np.roll(self.y, -length)
        self.y[-length:] = values
        self.t = np.roll(self.t, -length)
        self.t[-length:] = times

        # logging.debug(self.y)
        # logging.debug(self.t)
        
class TelemetryStream(object):
    # This is an abstract class and/or factory that provides a consistent interface across
    # vendors and devices.

    def __init__(self, *args, **kwargs):
        # Setup a specialized output logger
        self.logger = logging.getLogger()
        # self.logger.setLevel(logging.WARN)
        # Do anything else that would be generic across all monitor readers here
        self.update_funcs = []
        self.polling_interval = kwargs.get('polling_interval', 0.25)
        self.sampled_data_dur = kwargs.get('sampled_data_dur', 7)
        self.sampled_data = {}

        sampled_data_args = kwargs.get('values')
        # logging.debug(sampled_data_args)
        if sampled_data_args:
            for key, freq in zip(sampled_data_args[0::2], sampled_data_args[1::2]):
                self.sampled_data[key] = {'freq': int(freq),
                                          'samples': SampledDataBuffer(int(freq),
                                                                       self.sampled_data_dur)}

        # logging.debug('sampled data array')
        # logging.debug(self.sampled_data)

    def update_sampled_data(self, data):
        if not data:
            return

        for key, value in data.iteritems():
            if key in self.sampled_data.keys():
                t = data['timestamp']
                y = data[key]
                self.sampled_data[key]['samples'].rolling_append(t,y)

    def __del__(self):
        # Note that logging may no longer exist by here
        print("Tearing down connection object")
        self.close()

    def add_update_func(self, f):
        self.update_funcs.append(f)

    def run(self, blocking=False):
        # Create a main loop that just echoes the results to the loggers
        self.open()
        while 1:
            self.read(1, blocking=blocking)
            time.sleep(self.polling_interval)

    def open(self, *args, **kwargs):
        raise NotImplementedError

    def close(self, *args, **kwargs):
        raise NotImplementedError

    def read(self, *args, **kwargs):
        # Read should echo data to self.logger at "info" level
        raise NotImplementedError
    

class PhilipsTelemetryStream(TelemetryStream):
    """
    This class utilizes the data structures defined in IntellivueDecoder and
    the functions to communicate with the monitor via RS232.
    """

    # def __init__(self, serialPort, patientDirectory, selectedDataTypes):
    def __init__(self, *args, **kwargs):
        super(PhilipsTelemetryStream, self).__init__(*args, **kwargs)

        self.logger.name = 'PhilipsTelemetry'

        serialPort = kwargs.get('port')
        selectedDataTypes = kwargs.get('values')[::2]  # These come in as value, freq pairs; just need names

        self.port = serialPort
        self.rs232 = None  # This will be the socket object

        # Initialize Intellivue Decoder and Distiller
        self.decoder = IntellivueDecoder()
        self.distiller = IntellivueDistiller()

        # Initialize variables to keep track of time, and values to collect

        # Note: The listener automatically shuts down after this many seconds
        # Max is
        self.dataCollectionTime = 72 * 60 * 60  # seconds
        self.dataCollection = {'RelativeTime': self.dataCollectionTime * 8000}
        self.KeepAliveTime = 0
        self.messageTimes = []
        self.desiredWaveParams = {'TextIdLabel': selectedDataTypes}
        self.initialTime = 0
        self.relativeInitialTime = 0

        #  Initialize Messages
        self.AssociationRequest = self.decoder.writeData('AssociationRequest')
        self.AssociationAbort = self.decoder.writeData('AssociationAbort')
        self.ConnectIndication = {}
        self.AssociationResponse = ''
        self.MDSCreateEvent = {}
        self.MDSParameters = {}
        self.MDSCreateEventResult = ''
        self.MDSSetPriorityListWave = self.decoder.writeData('MDSSetPriorityListWAVE', self.desiredWaveParams)
        self.MDSSetPriorityListNumeric = ''
        self.MDSSetPriorityListResultWave = {}
        self.MDSSetPriorityListResultNumeric = {}
        self.MDSGetPriorityList = self.decoder.writeData('MDSGetPriorityList')
        self.MDSGetPriorityListResult = {}
        self.ReleaseRequest = self.decoder.writeData('ReleaseRequest')
        self.MDSExtendedPollActionNumeric = self.decoder.writeData('MDSExtendedPollActionNUMERIC',
                                                                   self.dataCollection)
        self.MDSExtendedPollActionWave = self.decoder.writeData('MDSExtendedPollActionWAVE', self.dataCollection)
        self.MDSExtendedPollActionAlarm = self.decoder.writeData('MDSExtendedPollActionALARM', self.dataCollection)
        self.KeepAliveMessage = self.decoder.writeData('MDSSinglePollAction')

        # Boolean to keep track of whether data should still be polled
        self.data_flow = False

        self.last_read_time = time.time()
        self.timeout = 10  # Seconds to wait before reset to transient failures

        self.last_keep_alive = time.time()


tstream = PhilipsTelemetryStream(port=4,
                                 values=["Pleth", 32*4, 'ECG', 64*4],
                                 polling_interval=0.05)

#
#opts.port = 4
#opts.values = ["Pleth", 32*4, 'ECG', 64*4]
#tstream = PhilipsTelemetryStream(port=opts.port,
#                                 values=opts.values,
#                                 polling_interval=0.05)

ModuleNotFoundError: No module named 'IntellivueDecoder'